In [ ]:
#Data Augmentation
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import CenterCrop
from google.colab.patches import cv2_imshow
from patchify import patchify

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(train_path, test_path):
    train_x = sorted(glob(os.path.join(train_path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(train_path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(test_path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(test_path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

def augment_data(images, masks, save_path, augment=True, format="same"):
    size = (512, 512)

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        
        name = x.split("/")[-1].split(".")[0]

   
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if format == "different":
          y = imageio.mimread(y)[0]
        else:
          y = cv2.imread(y, cv2.IMREAD_COLOR)


        x = cv2.resize(x, size)
        y = cv2.resize(y, size)

        if augment == True:
            aug = CenterCrop(height=320,width=448, always_apply=True, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]
            #print("\n",x1.shape,y1.shape)

            patches_img = patchify(x1,(64,64,3),step=32)
            #print(patches_img.shape)#(17,25,1,64,64,3): total patch created 8x8x1=64,of size:(64x64x3)
            #print(patches_img[0].shape)
            for i in range(patches_img.shape[0]):
              for j in range(patches_img.shape[1]):
                for k in range(patches_img.shape[2]):
                  all_patch_img = patches_img[i,j,k,:,:,:]
                  #cv2_imshow(all_patch_img)
                  tmp_image_name = f"{name}_{idx}{i}{j}.png"
                  image_path = os.path.join(save_path, "image",tmp_image_name)
                  cv2.imwrite(image_path, all_patch_img)

            patches_mask = patchify(y1,(64,64),step=32)
            #print(patches_mask.shape)#(8,8,64,64): total patch created 8x8=64,of size:(64x64)
            #print(patches_mask[0].shape)
            for i in range(patches_mask.shape[0]):
              for j in range(patches_mask.shape[1]):
                
                  all_patch_mask = patches_mask[i,j,:,:]
                  #cv2_imshow(all_patch_img)
                  tmp_mask_name = f"{name}_{idx}{i}{j}.png"
                  image_path = os.path.join(save_path, "mask",tmp_mask_name)
                  cv2.imwrite(image_path, all_patch_mask)      
        idx+=1

if __name__ == "__main__":

    np.random.seed(42)

    train_data_path = "/content/drive/MyDrive/Datasets/DRIVE"
    test_data_path = "/content/drive/MyDrive/Datasets/DRIVE"
    (train_x, train_y), (test_x, test_y) = load_data(train_data_path, test_data_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    """ Create directories to save the augmented data """
    create_dir("/content/drive/MyDrive/Augmented_data/train/image")
    create_dir("/content/drive/MyDrive/Augmented_data/train/mask/")
    create_dir("/content/drive/MyDrive/Augmented_data/test/image/")
    create_dir("/content/drive/MyDrive/Augmented_data/test/mask/")

    """ Data augmentation """
    augment_data(train_x, train_y, "/content/drive/MyDrive/Augmented_data/train/", augment=True, format="different")
    augment_data(test_x, test_y, "/content/drive/MyDrive/Augmented_data/test/", augment=True, format="different")

In [ ]:
#Making the data ready for training and Testing
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset

class DriveDataset(Dataset):
    def __init__(self, images_path, masks_path):

        self.images_path = images_path
        self.masks_path = masks_path
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = image/255.0
        image = np.transpose(image, (2, 0, 1))
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        """ Reading mask """
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = mask/255.0
        mask = np.expand_dims(mask, axis=0)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)

        return image, mask

    def __len__(self):
        return self.n_samples

In [ ]:
#Loss Function
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

In [ ]:
#Utils
import os
import time
import random
import numpy as np
import cv2
import torch

""" Seeding the randomness. """
def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

""" Create a directory. """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Calculate the time taken """
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
#RIMNet Model
import torch
import torch.nn as nn
import torch.nn.functional as F

class conv_block(nn.Module):
    def __init__(self,in_c,out_c, dropRate=0.0):
        super().__init__()
        self.conv1=nn.Conv2d(in_c, out_c,kernel_size=3,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU(inplace=True)
        self.droprate = dropRate
    def forward(self, inputs):
        x = self.relu(self.bn1(self.conv1(inputs)))
        if self.droprate > 0:
            x = F.dropout(x, p=self.droprate, training=self.training)
        return(x)

class res_block(nn.Module): #Residual Block
    def __init__(self, in_c, out_c, dropRate=0.0):
        super().__init__()
        self.conv1=conv_block(in_c,out_c,dropRate=0.0)
        self.conv2 =nn.Conv2d(out_c, out_c,kernel_size=3,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU(inplace=True)
        self.id = nn.Conv2d(in_c, out_c, kernel_size=1,padding=0)
        self.bn2= nn.BatchNorm2d(out_c)
        self.droprate = dropRate    
    def forward(self, inputs):
        x = self.bn1(self.conv2(self.conv1(inputs)))
        id = self.bn2(self.id(inputs))
        x = x + id
        x=self.relu(x)
        if self.droprate > 0:
            x = F.dropout(x, p=self.droprate, training=self.training)
        return x

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c,dropRate=0.0):
        super().__init__()
        self.conv = res_block(in_c, out_c,dropRate=dropRate)
        self.up = nn.ConvTranspose2d(out_c, out_c, kernel_size=2, stride=2, padding=0)
    def forward(self, inputs):
        x = self.conv(inputs)
        up = self.up(x)
       # print("Encoder output",x.shape,p.shape)
        return x, up

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c,dropRate=0.0):
        super().__init__()      
        self.conv1 = conv_block(in_c, out_c,dropRate=dropRate)
        self.pool = nn.MaxPool2d((2, 2))
        self.conv2 = res_block(out_c+out_c, out_c,dropRate=dropRate)
    def forward(self, inputs, skip):
        x = self.conv1(inputs)
        x = self.pool(x)
        x = torch.cat([x, skip], axis=1)
        x = self.conv2(x)
        return x

class build_unet(nn.Module):
    def __init__(self,dropRate=0.0):
        super().__init__()

        """ Encoder """
        self.e1 = encoder_block(3, 16,dropRate=dropRate)
        self.e2 = encoder_block(16, 32,dropRate=dropRate)
        self.e3 = encoder_block(32, 64, dropRate=dropRate)

        """ Bottleneck """
        self.b = conv_block(64, 128, dropRate=dropRate, )

        """ Decoder """
        self.d1 = decoder_block(128, 64,dropRate=dropRate )
        self.d2 = decoder_block(64, 32, dropRate=dropRate)
        self.d3 = decoder_block(32, 16, dropRate=dropRate)

        """ Classifier """
        self.outputs = nn.Conv2d(16, 1, kernel_size=1, padding=0)

    def forward(self, inputs):
        """ Encoder """
        c1, t1 = self.e1(inputs)
        c2, t2 = self.e2(t1)
        c3, t3 = self.e3(t2)
      
        """ Bottleneck """
        b = self.b(t3)
        
        """ Decoder """
        d1 = self.d1(b, c3)
        d2 = self.d2(d1, c2)
        d3 = self.d3(d2, c1)
      
        outputs = self.outputs(d3)

        return outputs

if __name__ == "__main__":
    x = torch.randn((1, 3, 64,64))
    f = build_unet(dropRate=0.08)
    y = f(x)
    print("output",y.shape)
    

output torch.Size([1, 1, 64, 64])


In [ ]:
#Training - Validation - Testing 
import os
import time
from glob import glob

from operator import add, sub
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import random
import csv

import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torch.nn as nn

from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score,confusion_matrix,roc_auc_score

# This function lodes the traning dataset and divide it into two non-overlapping validation and traning datasets

def get_train_valid_loader(data_dir,
                           batch_size,
                           random_seed,
                           kfolde=10,
                           shuffle=True,
                           show_sample=False,
                           num_workers=1,
                           pin_memory=False,
                           shuffelthevaluditation=1):
    """
 
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - kfolde: the number of foldes in cross validation
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot  sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    -shuffelthevaluditation: the folding index of validation set
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """


    # load the dataset
    train_x = sorted(glob(os.path.join(data_dir,  "image", "*")))[:2106]
    train_y = sorted(glob(os.path.join(data_dir, "mask", "*")))[:2106]

    valid_x = sorted(glob(os.path.join(data_dir, "image", "*")))[2106:]
    valid_y = sorted(glob(os.path.join(data_dir, "mask", "*")))[2106:]
    
    
    train_dataset = DriveDataset(train_x, train_y)
    valid_dataset = DriveDataset(valid_x, valid_y)
   
    dataset=ConcatDataset([train_dataset, valid_dataset])
    
    valid_size=(len(dataset)/kfolde)/(len(dataset))
    num_train = len(dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[:split* (int( shuffelthevaluditation) -1 )] + indices[split*int( shuffelthevaluditation):], indices[split * (int( shuffelthevaluditation) -1 ):split * int( shuffelthevaluditation)]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    data_str = f"Dataset Size:\nTrain sampler: {len(train_sampler)} - Validsampler: {len(valid_sampler)}\n"
    
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    
    valid_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    ) 
    
    return (train_loader, valid_loader)

#best val accuracy
#written to CSV file
#K fold Cross Validation

def calculate_metrics(y_true, y_pred):
    """ Ground truth """
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    """ Prediction """
    y_pred = y_pred.cpu()
    y_pred = y_pred.detach().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    #score_recall = recall_score(y_true, y_pred)
    #score_precision = precision_score(y_true, y_pred)
    #score_acc = accuracy_score(y_true, y_pred)
    cm1=confusion_matrix(y_true,y_pred)
    total1=sum(sum(cm1))
    score_recall= cm1[0,0]/(cm1[0,0]+cm1[0,1])
    score_precision= cm1[0,0]/(cm1[0,0]+cm1[1,0])
    score_acc = (cm1[0,0]+cm1[1,1])/total1
    Specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    Sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])

    auc=roc_auc_score(y_true, y_pred)
    intersection = np.sum(y_true * y_pred)
    dice_coeff=(2*intersection +1) / (np.sum(y_true) + np.sum(y_pred)+ 1)


    return [score_jaccard, score_f1, score_recall, score_precision, score_acc,Specificity, Sensitivity,auc,dice_coeff]

def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0
    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0,0.0,0.0,0.0,0.0]
    model.train()
    for x, y in loader:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        score = calculate_metrics(y, y_pred)
        metrics_score = list(map(add, metrics_score, score))
    epoch_loss = epoch_loss/len(loader)
    metrics_score = [sc / len(loader) for sc in metrics_score]
    
    return epoch_loss, metrics_score
    #epoch_loss = epoch_loss/len(loader)
    #return epoch_loss,

def evaluate(model, loader, loss_fn, device,mode):
    epoch_loss = 0.0
    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0,0.0,0.0,0.0,0.0]
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()
            score = calculate_metrics(y, y_pred)
            metrics_score = list(map(add, metrics_score, score))
        epoch_loss = epoch_loss/len(loader)
        metrics_score = [sc / len(loader) for sc in metrics_score]
    return epoch_loss, metrics_score

def test(model, loader, loss_fn, device,mode):
    epoch_loss = 0.0
    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0,0.0,0.0,0.0,0.0]
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()
            score = calculate_metrics(y, y_pred)
            metrics_score = list(map(add, metrics_score, score))
        epoch_loss = epoch_loss/len(loader)
        metrics_score = [sc / len(loader) for sc in metrics_score]
    return epoch_loss, metrics_score



if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Directories """
    create_dir("/content/drive/MyDrive/Program/files")

    """ Load test dataset """
    data_dir="/content/drive/MyDrive/Augmented_data/train/"
    test_x = sorted(glob("/content/drive/MyDrive/Augmented_data/test/image/*"))
    test_y = sorted(glob("/content/drive/MyDrive/Augmented_data/test/mask/*"))
    
    """ Hyperparameters """
    num_epochs=2
    batch_size=2
    k=10
    lr = 1e-3
    checkpoint_path= "/content/drive/MyDrive/Program/files/checkpoint64.pth"
   
    """ Dataset and loader """
    
    train_loader, valid_loader=get_train_valid_loader(data_dir=data_dir,
                                                      batch_size=batch_size,
                                                      random_seed=False,
                                                      kfolde=k,
                                                      shuffle=True,
                                                      show_sample=False,
                                                      num_workers=1,
                                                      pin_memory=False,
                                                      shuffelthevaluditation=1)
    print("train data loader",len(train_loader))
    print("valid data loader",len(valid_loader)) 
    
    test_dataset=DriveDataset(test_x,test_y)
    test_loader = DataLoader(
                              dataset=  test_dataset,
                             batch_size=batch_size,
                              shuffle=False,
                              num_workers=1
                         )

    loaded_checkpoint=torch.load(checkpoint_path)
    #epoch=loaded_checkpoint["epoch"]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = build_unet(dropRate=0.09)
    model = model.to(device)
    print('Number of model parameters: {}'.format(
      sum([p.data.nelement() for p in model.parameters()])))  
        
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

    model.load_state_dict(loaded_checkpoint["model_state"])#loading the trained model
    optimizer.load_state_dict(loaded_checkpoint["optim_state"])

    loss_fn = DiceBCELoss().to(device)

    #creating dictionary to save checkpoints
    checkpoint={
              #"epoch":10,
              "model_state":model.state_dict(),
              "optim_state":optimizer.state_dict()
                }

    """oneing/creating required csv files for recording data"""
       
    with open('/content/drive/MyDrive/TrainResult/train_64_512.csv','a+') as td:
        training_details=csv.writer(td)
        with open('/content/drive/MyDrive/TrainResult/val_64_512.csv','a+') as vd:
            val_details=csv.writer(vd)
            with open('/content/drive/MyDrive/TestResult/Test_64_512.csv','a+') as tsd:
                Test_details=csv.writer(tsd)
                """ Training the model """
                best_valid_loss = float("inf")                  
                best_train_accuracy=0.0
                best_val_accuracy=0.0
                best_test_accuracy=0.0
                best_auc=0.0
                best_sp=0.0
                best_se=0.0
                best_f1=0.0
                best_iou=0.0
                best_dc=0.0
                              

                for epoch in range(num_epochs):
                    start_time = time.time()

                    train_loss, train_score= train(model, train_loader, optimizer, loss_fn, device)
                    print("Training Done")
                    valid_loss, val_score = evaluate(model, valid_loader, loss_fn, device,'valid')
                    print("Validation Done")
                    test_loss, test_score = test( model,test_loader, loss_fn, device,'Test')
                    print("Test Done")
                    """ Saving the model """
                    if valid_loss < best_valid_loss:
                      data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
                      print(data_str)

                      best_valid_loss = valid_loss
                      torch.save(checkpoint, checkpoint_path)#saving the checkpoiunts
                                  
                    if best_train_accuracy< train_score[4]:
                      best_train_accuracy = max(train_score[4],best_train_accuracy)                    
                      print(f"Best train Accuracy:{best_train_accuracy:1.4f}")
                    if best_val_accuracy < val_score[4]:
                      best_val_accuracy = max(val_score[4],best_val_accuracy)                    
                      print(f"Best Validation Accuracy:{best_val_accuracy:1.4f}")
                    if best_test_accuracy < test_score[4]:
                      best_test_accuracy = max(test_score[4],best_test_accuracy)                    
                      print(f"Best Test Accuracy:{best_test_accuracy:1.4f}")
                    if best_auc < test_score[7]:
                      best_auc = max(test_score[7],best_auc) 
                    if best_sp < test_score[2]:
                      best_sp = max(test_score[2],best_sp) 
                    if best_se < test_score[6]:
                      best_se = max(test_score[6],best_se) 
                    if best_f1 < test_score[1]:
                      best_f1= max(test_score[1],best_f1) 
                    if best_iou < test_score[0]:
                      best_iou = max(test_score[0],best_iou) 
                    if best_dc < test_score[8]:
                      best_dc = max(test_score[8],best_dc) 

                     

                    end_time = time.time()
                    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
                                  
                    data_str = f'Epoch: {epoch+1+11:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
                    data_str += f'\tTrain Loss: {train_loss:.3f}'
                    data_str += f'\tVal. Loss: {valid_loss:.3f}'
                    data_str += f'\tTest. Loss: {test_loss:.3f}'
                    print(data_str)

                    
                    t_jaccard = train_score[0]
                    t_f1 = train_score[1]
                    t_recall = train_score[2]
                    t_precision = train_score[3]
                    t_acc = train_score[4]
                    t_sp=train_score[5]
                    t_se=train_score[6]
                                                        
                    jaccard = val_score[0]
                    f1 = val_score[1]
                    recall = val_score[2]
                    precision = val_score[3]
                    acc = val_score[4]
                    sp=val_score[5]
                    se=val_score[6]

                                  
                    training_details.writerow([str(epoch+1+11),float(train_loss),float(t_jaccard),float(t_f1),float(t_recall),float(t_precision),float(t_acc),float(t_sp),float(t_se)])                    
                    val_details.writerow([str(epoch+1+11),float(valid_loss),float(jaccard),float(f1),float(recall),float(precision),float(acc),float(sp),float(se)])                                  
                    Test_details.writerow([str(epoch+1+11),float(test_loss),float(test_score[0]),float(test_score[1]),float(test_score[2]),float(test_score[3]),float(test_score[4]),float(test_score[5]),float(test_score[6]),float(test_score[7]),float(test_score[8])])                                           
                    print(f"\n\ttr_acc: {t_acc:1.4f}\tval_acc: {acc:1.4f}\tTest_acc: {test_score[4]:1.4f}")
                    print(f"\n\tValidation_Jaccard: {jaccard:1.4f} \tF1: {f1:1.4f} \tRecall: {recall:1.4f} \tPrecision: {precision:1.4f} \tSP: {sp:1.4f}\tSE: {se:1.4f}")
                    print(f"\n\tTest_Jaccard: {test_score[0]:1.4f} \tF1: {test_score[1]:1.4f} \tRecall: {test_score[2]:1.4f} \tPrecision: {test_score[3]:1.4f} \tSP: {test_score[5]:1.4f}\tSE: {test_score[6]:1.4f}\tAUC: {test_score[7]:1.4f}\tDC: {test_score[8]:1.4f}")

                    train_loader, valid_loader =get_train_valid_loader(kfolde=k,shuffelthevaluditation=(epoch%k)+1,random_seed=False,data_dir=data_dir,batch_size=batch_size, shuffle=True)
        print("Completed Successfully")
        print(f"Best train Accuracy:{best_train_accuracy:1.4f}")
        print(f"Best Validation Accuracy:{best_val_accuracy:1.4f}")
        print(f"Best Test Accuracy:{best_test_accuracy:1.4f}")
        print(f"Best Test IOU:{best_iou:1.4f}")
        print(f"Best Test F1-Score:{best_f1:1.4f}")
        print(f"Best Test Sensitivity:{best_se:1.4f}")
        print(f"Best Test Specificity:{best_sp:1.4f}")
        print(f"Best Test AUC:{best_auc:1.4f}")
        print(f"Best Test DC:{best_dc:1.4f}")



train data loader 1053
valid data loader 117
Number of model parameters: 6760833
Training Done
Validation Done


KeyboardInterrupt: ignored